In [11]:
#@title Day 1-1 Trebuchet?!
from google.colab import files
with open('2023-1-1.txt','r') as f:
  lines = f.read().splitlines()

digits = [[i for i in l if '0' <= i <= '9'] for l in lines]
values = list(map(int, [d[0]+d[-1] for d in digits]))
print (sum(values))

54573
